# Deep Learning Neural Network on Animal Shelter data

In [1]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import theano

import matplotlib.pyplot as plt
import os


Using Theano backend.


In [3]:
# Set up my data directories from different machines

mac_data_dir = '/Users/christopherallison/Documents/Coding/Data'
linux_data_dir = '/home/chris/data'
win_data_dir = u'C:\\Users\\Owner\\Documents\\Data'

# Set data directory for example

data_dir = mac_data_dir

In [4]:
# Load our prepared dataset and reference data

df = pd.read_csv(os.path.join(data_dir, "prepared_animals_df.csv"),index_col=0)

In [5]:
df.head()

,OutcomeType,AnimalType,AgeuponOutcome,Color,Intact,Gender,NameLength,BreedKMeans
0,Return_to_owner,1,1,0,0,1,7,0
1,Euthanasia,0,1,1,0,0,5,1
2,Adoption,1,2,2,0,1,6,3
3,Transfer,0,0,2,1,1,3,1
4,Transfer,1,2,3,0,1,3,0


In [6]:
# Drop uneccesary columns
X = df.drop('OutcomeType', axis=1)
X.dtypes

AnimalType        int64
AgeuponOutcome    int64
Color             int64
Intact            int64
Gender            int64
NameLength        int64
BreedKMeans       int64
dtype: object

In [7]:
# We now have a dataframe with 7 features.

X.head()

,AnimalType,AgeuponOutcome,Color,Intact,Gender,NameLength,BreedKMeans
0,1,1,0,0,1,7,0
1,0,1,1,0,0,5,1
2,1,2,2,0,1,6,3
3,0,0,2,1,1,3,1
4,1,2,3,0,1,3,0


In [8]:
X = X.as_matrix()

In [9]:
outcomes = df.OutcomeType.unique()

In [10]:
from sklearn import preprocessing

# This code takes our text labels and creates an encoder that we use
# To transform them into an array

encoder = preprocessing.LabelEncoder()
encoder.fit(outcomes)

encoded_y = encoder.transform(outcomes)
encoded_y

#We can also inverse_transform them back.
list(encoder.inverse_transform([0, 1, 2, 3, 4]))

#We still need to transform the array into a matrix - this is called one hot encoding. It allows us to track the probability of each possible outcome separately.
# convert integers into dummy variables (one hot encoding)

from keras.utils import np_utils

# We'll practice with the dummy for now, but we'll transform the full labels later.
dummy_y = np_utils.to_categorical(encoded_y)
dummy_y

#We'll transform the labels into their array value, though.
df.OutcomeType = encoder.transform(df.OutcomeType)

In [11]:
train_target = np_utils.to_categorical(df['OutcomeType'].values)
train_target

array([[ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [12]:
model = Sequential()
model.add(Dense(5, input_dim=7, init='normal', activation="relu"))
model.add(Dense(5, init='normal', activation='sigmoid'))

# Compile model
print("Compiling model...")
model.compile(loss='categorical_crossentropy', optimizer='adam',
             metrics=['accuracy'])

Compiling model...


In [13]:
hist = model.fit(X, train_target, validation_split=0.2)
print("")
print(hist.history)

Train on 21383 samples, validate on 5346 samples
Epoch 1/10
21383/21383 [==============================] - 0s - loss: 1.2928 - acc: 0.3991 - val_loss: 1.2043 - val_acc: 0.4027
Epoch 2/10
21383/21383 [==============================] - 0s - loss: 1.0999 - acc: 0.5488 - val_loss: 1.0308 - val_acc: 0.5924
Epoch 3/10
21383/21383 [==============================] - 0s - loss: 1.0091 - acc: 0.5948 - val_loss: 1.0014 - val_acc: 0.5965
Epoch 4/10
21383/21383 [==============================] - 0s - loss: 0.9933 - acc: 0.5893 - val_loss: 0.9886 - val_acc: 0.5875
Epoch 5/10
21383/21383 [==============================] - 0s - loss: 0.9843 - acc: 0.5922 - val_loss: 0.9876 - val_acc: 0.5857
Epoch 6/10
21383/21383 [==============================] - 0s - loss: 0.9787 - acc: 0.5907 - val_loss: 0.9763 - val_acc: 0.5907
Epoch 7/10
21383/21383 [==============================] - 0s - loss: 0.9744 - acc: 0.5935 - val_loss: 0.9769 - val_acc: 0.5988
Epoch 8/10
21383/21383 [==============================] - 0s -

In [14]:
model.evaluate(X, train_target)

25728/26729 [===========================>..] - ETA: 0s

[0.96499712192283937, 0.59575741704213436]

In [17]:
model.predict(X[0:2], verbose=1)

2/2 [==============================] - 0s


array([[  1.65767148e-01,   1.28618485e-04,   2.72578606e-03,
          5.14156856e-02,   3.92738879e-02],
       [  2.49545947e-01,   3.18642217e-03,   1.03837708e-02,
          2.35910490e-02,   1.30494133e-01]])